In [1]:
# ipython settings
%load_ext autoreload
%autoreload 2
import warnings

warnings.filterwarnings('ignore')

# 3. Synthetic experiments with Symbolic Pursuit

In this notebook, we shall reproduce one of the experiments from Section 6.1 of the paper.
The idea is to start with a linear pseudo black-box for which the importance vector is known unambiguously and see which interpretability methods identifies this vector the most precisely. Let us start by the useful imports.


In [2]:
from symbolic_pursuit.models import SymbolicRegressor  # our symbolic model class
from sklearn.metrics import mean_squared_error # we are going to assess the quality of the model based on the generalization MSE
from sympy import init_printing # We use sympy to display mathematical expresssions 
import numpy as np # we use numpy to deal with arrays
import lime 
import lime.lime_tabular
init_printing()

We now define a linear pseudo black-box $f$ defined on a 3 dimensional feature space.

$$ f(x_1,x_2,x_3)= x_1 + 2 \cdot x_2 + 3 \cdot x_3$$ 

The importance vector associated to this model is trivially given by $\beta = (1,2,3)$ In this case, we shall keep it unnormalised, unlike in the main paper as we deal with few examples. Let us translate this in Python. 

In [3]:
def f(X):
    return -1*X[:, 0]**2 + 2*X[:,1]**2 + 3*X[:,2]**3

dim_X = 3

Now draw uniformly 100 test points  that we will feed to a *LIME* explainer <cite data-cite="2480681/WCEBQ7N9"></cite> and to train a Symbolic model.

In [4]:
n_pts = 100
X = np.random.uniform(0, 1, (n_pts, dim_X))

Now we draw 10 test ponits $x_{test} \equiv U([0,1]^3)$ that we are going to use in order to evaluate the perfomances of both explainers on unseen data.

In [5]:
n_test = 10
X_test = np.random.uniform(0, 1, (n_test, dim_X))

Since LIME produces importance vectors with entries in the form $(feature \ domain , importance)$ for each feature appearing in decreasing order of importance, we implement a function which identifies the feature from the first entry of the tuple and who sorts the importances in the form $(importance(x_1), importance(x_2), importance(x_3))$.

In [6]:
def order_weights(exp_list):
    ordered_weights = [0 for _ in range(dim_X)]
    for tup in exp_list:
        feature_id = int(tup[0].split('x_')[1][0])
        ordered_weights[feature_id-1] = tup[1]    
    return ordered_weights    

We are now ready to extract the feature importance for our 10 test points as predicted by the LIME explainer :

In [7]:
lime_weight_list = []
explainer = lime.lime_tabular.LimeTabularExplainer(X, 
                                                   feature_names=["x_"+str(k) for k in range(1,dim_X+1)], 
                                                   class_names=['f'], 
                                                   verbose=True,
                                                   mode='regression')

for i in range(n_test):
    exp = explainer.explain_instance(X_test[i], f, num_features=dim_X)
    lime_weight_list.append(order_weights(exp.as_list()))  
                            
print(lime_weight_list)    

Intercept 0.7405264655100314
Prediction_local [2.20310887]
Right: 2.608365057349564
Intercept 1.4583758896288619
Prediction_local [0.12329823]
Right: 0.1560636050246768
Intercept 1.4268850931161419
Prediction_local [0.08905213]
Right: 0.13488257824282124
Intercept 1.683486209606011
Prediction_local [-0.6447336]
Right: -0.3279022373241939
Intercept 1.668153206984909
Prediction_local [-0.53783986]
Right: -0.6690314925412343
Intercept 1.1622619429446956
Prediction_local [0.93104246]
Right: 1.4477905821351933
Intercept 1.4372498635832134
Prediction_local [0.16911096]
Right: 0.062496300810271424
Intercept 0.940577093371443
Prediction_local [1.59609345]
Right: 1.4558204845521794
Intercept 0.9546226090342119
Prediction_local [1.56107679]
Right: 2.476292685595626
Intercept 1.4468405952230627
Prediction_local [0.07945896]
Right: 0.13707143587208612
[[0.23652031125435852, 1.1223741632087427, 0.10368793096238493], [0.18417919046996042, -0.42944573657422985, -1.0898111140824274], [-0.6009886795480

As we can see from the last output, which is the list of predicted importance vectors, LIME seems to produce a big variety of importance vectors. This is suprising for a global linear model. We also note that the relative importance seem inconsistent with the true importance vector $\beta$ defined above. Let us now train a Symbolic model for $f$ based on our training set.

In [8]:
symbolic_model = SymbolicRegressor(maxiter=20,
                 eps=1.0e-4)
symbolic_model.fit(f, X)

Model created with the following hyperparameters :
 loss_tol=0.001 
 ratio_tol=0.9 
 maxiter=20 
 eps=0.0001 
 random_seed=42
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Now working on term number  1 .
Now working on hyperparameter tree number  1 .
         Current function value: 0.270213
         Iterations: 3
         Function evaluations: 304
         Gradient evaluations: 36
Now working on hyperparameter tree number  2 .
         Current function value: 0.886529
         Iterations: 1
         Function evaluations: 211
         Gradient evaluations: 20
Now working on hyperparameter tree number  3 .
         Current function value: 0.406623
         Iterations: 2
         Function evaluations: 332
         Gradient evaluations: 29
The tree number  1  was selected as the best.
Backfitting complete.
The current model has the following expression:  0.999997804137333*[ReLU(P1)]**0.249997176414374*exp(-0.250000636481756*I*pi)*bess

We now ask our symbolic model to predict the importance vectors for each test point.

In [9]:
symbolic_weight_list = [] 
for k in range(n_test):
    symbolic_weight_list.append(symbolic_model.get_feature_importance(X_test[k]))
    

In [10]:
print(symbolic_weight_list)

[[0.0711919975428705, -0.0196895643913308, 0.0928001247198033], [0.315262701348832, -0.0876304448834403, 0.411055138860024], [0.0631766231831720, -0.0174583601419887, 0.0823485095099213], [0.0780174719311039, -0.0215895414573272, 0.101700174640042], [0.0932499535578633, -0.0258297398820618, 0.121562507889874], [0.0680905869678336, -0.0188262384699901, 0.0887560528544733], [0.136433073664228, -0.0378504337083696, 0.177870963946919], [0.204552826446150, -0.0568126274937737, 0.266695442045996], [0.0195224861073649, -0.00530655140868202, 0.0254258727669825], [0.127620469656256, -0.0353973081738693, 0.166379804338517]]


As we can see, our results appear to be always consistent and very close to the true importance vector $\beta$.

In [11]:
print (np.linalg.norm(symbolic_model.predict(X_test) - f(X_test)))

3.5334769433466486


In [12]:
symbolic_model.predict(X_test)

array([0.54751   , 0.41773502, 0.55061806, 0.54469921, 0.53795729,
       0.54873852, 0.51631113, 0.47855498, 0.56262748, 0.52095646])

In [13]:
f(X_test)

array([ 2.60836506,  0.15606361,  0.13488258, -0.32790224, -0.66903149,
        1.44779058,  0.0624963 ,  1.45582048,  2.47629269,  0.13707144])

## References<div class="cite2c-biblio"></div>

<div class="cite2c-biblio"></div>